In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('../data/pc/train.csv',index_col=None)
df.drop(['assessed_2019','building_value_2019','land_value_2019'],axis=1,inplace=True)

In [3]:
X = df.drop(columns=['acct','TARGET'])
y = df['TARGET']

# Ridge ($\ell_2$)

In [13]:
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

rmse_scores = {}

for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X,y)

    y_pred = ridge.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred))

    rmse_scores[a] = rmse

/home/sethkaufman7/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.08572e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sethkaufman7/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.33299e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sethkaufman7/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.69939e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sethkaufman7/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.80168e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", over

In [14]:
rmse_scores

{0.01: np.float64(42107.305615080055),
 0.1: np.float64(42101.37720001765),
 1.0: np.float64(42102.34097079095),
 10.0: np.float64(42103.37436020412),
 100.0: np.float64(42106.19035137568)}

## $\lambda = 0.1$

In [15]:
ridge = Ridge(alpha=0.1)
ridge.fit(X,y)

/home/sethkaufman7/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.33299e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(alpha=0.1)

In [16]:
y_pred = ridge.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Train RMSE (Ridge): {rmse:.4f}")

Train RMSE (Ridge): 42101.3772


In [17]:
test = pd.read_csv('../data/pc/test.csv',index_col=None)

In [18]:
test = test.drop_duplicates(subset='acct')

In [20]:
X_test = test.drop(columns=['acct'])  # or drop other non-feature cols
acct_ids = test['acct']               # store acct ID

In [21]:
y_pred = ridge.predict(X_test)

In [22]:
output_df = pd.DataFrame({
    'acct': acct_ids,
    'TARGET': y_pred
})

In [23]:
output_df.rename({'acct':'ACCOUNT'},axis=1,inplace=True)

In [25]:
output_df.reset_index(inplace=True,drop=True)

In [26]:
output_df

,ACCOUNT,TARGET
0,bb75f25168addc1117840b10c0fd6cd0c2a7b7c6,372597.540283
1,8def0ccceda200b673872a8a9367644767989f3b,140216.999359
2,ca33e57b3b13e843909f4b6cbd9a3410387bd45a,274875.499726
3,3e0f6f6090a8226ce67ccf2f8630b8ad630b8d55,161327.027440
4,63facf82adbae10b23f7fabc93188c95bd832f51,303217.456451
...,...,...
418853,24847d36c333ab3376848ee1bda74916286a2a4b,346633.054826
418854,62ba19f1655097ac9cc8682aec435d22a653bfa0,349625.233483
418855,552b5300d14369d8fd792ea6228dfa683014b2f5,447081.888710
418856,290d351d16d00e79e89dc5404412f07769f40038,297169.981041


In [27]:
output_df.to_csv('../data/pc/predictions_2.csv', index=False)

## Ridge CV

In [4]:
# Try a range of alpha values
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

In [5]:
# Ridge with cross-validation (default 5-fold)
ridge_cv = RidgeCV(alphas=alphas, scoring='neg_root_mean_squared_error')

In [6]:
ridge_cv.fit(X, y)

: 

In [ ]:

# Best alpha
print(f"Best alpha (Ridge): {ridge_cv.alpha_}")

# RMSE on training data
y_pred = ridge_cv.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Train RMSE (Ridge): {rmse:.4f}")

In [ ]:
test = pd.read_csv('../data/pc/test.csv',index_col=None)

In [ ]:
test = test.drop_duplicates(subset='acct')

In [ ]:
X_test = test.drop(columns=['acct'])  # or drop other non-feature cols
acct_ids = test['acct']               # store acct ID

In [ ]:
y_pred = ridge.predict(X_test)

In [ ]:
output_df = pd.DataFrame({
    'acct': acct_ids,
    'TARGET': y_pred
})

In [ ]:
output_df.rename({'acct':'ACCOUNT'},axis=1,inplace=True)

In [ ]:
output_df.reset_index(inplace=True,drop=True)

In [ ]:
output_df

,ACCOUNT,TARGET
0,bb75f25168addc1117840b10c0fd6cd0c2a7b7c6,372597.540283
1,8def0ccceda200b673872a8a9367644767989f3b,140216.999359
2,ca33e57b3b13e843909f4b6cbd9a3410387bd45a,274875.499726
3,3e0f6f6090a8226ce67ccf2f8630b8ad630b8d55,161327.027440
4,63facf82adbae10b23f7fabc93188c95bd832f51,303217.456451
...,...,...
418853,24847d36c333ab3376848ee1bda74916286a2a4b,346633.054826
418854,62ba19f1655097ac9cc8682aec435d22a653bfa0,349625.233483
418855,552b5300d14369d8fd792ea6228dfa683014b2f5,447081.888710
418856,290d351d16d00e79e89dc5404412f07769f40038,297169.981041


# LASSO ($\ell_1$)

In [4]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(alphas=[0.001, 0.01, 0.1, 1.0, 10], cv=5)
lasso_cv.fit(X, y)

print(f"Best alpha (Lasso): {lasso_cv.alpha_}")
print(f"Non-zero features: {(lasso_cv.coef_ != 0).sum()} / {len(lasso_cv.coef_)}")

y_pred = lasso_cv.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Train RMSE (Lasso): {rmse:.4f}")

: 